# Day 18: Operation Order

As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework.

Unfortunately, it seems like this "math" follows different rules than you remember.

The homework (your puzzle input) consists of a series of expressions that consist of addition (+), multiplication (*), and parentheses ((...)). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence, and are evaluated left-to-right regardless of the order in which they appear.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are as follows:

1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
      9   + 4 * 5 + 6
         13   * 5 + 6
             65   + 6
                 71

Parentheses can override this order; for example, here is what happens if parentheses are added to form 1 + (2 * 3) + (4 * (5 + 6)):

1 + (2 * 3) + (4 * (5 + 6))
1 +    6    + (4 * (5 + 6))
     7      + (4 * (5 + 6))
     7      + (4 *   11   )
     7      +     44
            51

Here are a few more examples:

    2 * 3 + (4 * 5) becomes 26.
    5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 437.
    5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 12240.
    ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 13632.

Before you can help with the homework, you need to understand it yourself. Evaluate the expression on each line of the homework; what is the sum of the resulting values?

In [1]:
# Python imports
from dataclasses import dataclass
from operator import mul, add
from pathlib import Path
from typing import List, Optional

We load the input data by stripping out any confusing whitespace.

In [2]:
def load_data(fpath):
    with Path(fpath).open("r") as ifh:
        return [_.strip().replace(" ","") for _ in ifh.readlines()]

To solve the first puzzle, we read each line from left to right, and keep a running total. We update that total with each successive operation - whether it's addition or multiplication.

To handle parentheses, we recurse into the same function with the remainder of the line that's left to be parsed, when we hit a left parenthesis `(`. We pop back up when we hit a right parenthesis `)` (or the end of the line) with the running total and the remainder of the line.

In [3]:
def parse_line(line):
    tot = 0
    op = None
    line = list(line)
    while len(line):
        char = line.pop(0)
        if char == "(":
            val, line = parse_line(line)
            if tot == 0:
                tot = val
            else:
                tot = op(tot, val)
        elif char == ")":
            return tot, line
        elif char == "+":
            op = add
        elif char == "*":
            op = mul
        elif char.isdigit():
            if tot == 0:
                tot = int(char)
            else:
                tot = op(tot, int(char))
    return tot, line

Testing this out on the test data:

In [4]:
lines = load_data("day18_test.txt")
for line in lines:
    print(line)
    print(parse_line(line))

2*3+(4*5)
(26, [])
5+(8*3+9+3*4*3)
(437, [])
5*9*(7*3*3+9*3+(8+6*4))
(12240, [])
((2+4*9)*(6+9*8+6)+6)+2+4*2
(13632, [])


It seems to work, so let's sum the results for the first puzzle:

In [5]:
lines = load_data("day18_test.txt")
sum(parse_line(line)[0] for line in lines)

26335

And now the second puzzle:

In [6]:
lines = load_data("day18_data.txt")
sum(parse_line(line)[0] for line in lines)

3647606140187

## Part Two

You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: advanced math.

Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated before multiplication.

For example, the steps to evaluate the expression 1 + 2 * 3 + 4 * 5 + 6 are now as follows:

1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
  3   *   7   * 5 + 6
  3   *   7   *  11
     21       *  11
         231

Here are the other examples from above:

    1 + (2 * 3) + (4 * (5 + 6)) still becomes 51.
    2 * 3 + (4 * 5) becomes 46.
    5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 1445.
    5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 669060.
    ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 23340.

What do you get if you add up the results of evaluating the homework problems using these new rules?

I suspect that what the puzzle setter wants us to do is build a grammar parser that can handle parentheses and multiplications, and manage the swapping of priorities. However, that's quite hard and defeated me in the time I was willing to spend.

In Python we can hack together a solution that uses Python's own grammar-parsing to cope with parentheses (the bit that beat me), so long as we can swap multiplication and addition. Swapping operations is quite straightforward - we implement a class `MyInt` with those two operations reversed:

In [7]:
class MyInt:
    
    def __init__(self, val: int):
        self.val = val
        
    def __mul__(self, val):
        return MyInt(int(self.val) + int(val))
    
    def __add__(self, val):
        return MyInt(int(self.val) * int(val))
    
    def __int__(self):
        return int(self.val)
    
    def __str__(self):
        return f"{self.val}"
    
    def __repr__(self):
        return f"{self.val}"

Now multiplication is addition:

In [8]:
MyInt(9) * MyInt(15)

24

And addition is multiplication:

In [9]:
MyInt(9) + 2

18

But we need to convert all the numbers in the input line into `MyInt` instances - and then we can evaluate the line with `eval()` (which is a security risk but, hey!)

In [10]:
def eval_line(line):
    newline = []
    for char in line.strip().replace(" ",""):
        if char.isdigit():
            newline.append(f"MyInt({char})")
        elif char == "*":
            newline.append("+")
        elif char == "+":
            newline.append("*")
        else:
            newline.append(char)
#     print("".join(newline))
    return int(eval("".join(newline)))
            

We test out the function:

In [11]:
eval_line("1 + 2 * 3 + 4 * 5 + 6")

231

It seems to work, so let's try it with the test input:

In [12]:
lines = load_data("day18_test.txt")
for line in lines:
    print(line, eval_line(line))

2*3+(4*5) 46
5+(8*3+9+3*4*3) 1445
5*9*(7*3*3+9*3+(8+6*4)) 669060
((2+4*9)*(6+9*8+6)+6)+2+4*2 23340


That's fine, so let's calculate the answer to the second puzzle:

In [13]:
lines = load_data("day18_data.txt")
sum([eval_line(line) for line in lines])

323802071857594